# Milestone 4

In [1]:
import pandas as pd

In [2]:
from pyspark.sql import SparkSession, Row
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, GBTClassifier
from pyspark.ml.classification import MultilayerPerceptronClassifier, LinearSVC, OneVsRest
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [ ]:
#!pip install h2o

In [4]:
# import h2o
# from h2o.automl import H2OAutoML

In [3]:
###### Define train-test split ######
train_split = 0.8
test_split = 0.2

###### Define target label ######
target_column = 'Delay'

In [4]:
###
#for whole dataset
##### Set up session & data ######
#Initialize Spark session
spark = SparkSession.builder.appName("DecisionTreeJob").getOrCreate()

#Import the data as an RDD
#csv_rdd = spark.sparkContext.textFile("gs://final-bucket-jy/balanced_flight_encoded.csv/part-00197-1a134fbf-5193-4064-89a5-cb01a8fb1d12-c000.csv")
csv_rdd = spark.sparkContext.textFile("gs://final-bucket-jy/balanced_flight_encoded.csv")


#Convert the RDD to a DataFrame
csv_rows = csv_rdd.map(lambda line: line.split(","))
header = csv_rows.first()  # Extract header
csv_data = csv_rows.filter(lambda row: row != header).map(lambda row: Row(**{header[i]: float(row[i]) for i in range(len(header))}))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/21 10:18:26 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
24/04/21 10:18:26 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
24/04/21 10:18:26 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
24/04/21 10:18:26 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
24/04/21 10:18:36 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #1,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at com.google.common.util.concurrent.FluentFuture$TrustedFuture.get(FluentFuture.java:88)
	at org.apache.hadoop.util.concurrent.ExecutorHelper.logThrowableFromAfterExecute(ExecutorHelper.java:48)
	at org.apache.hadoop.util.concurrent.HadoopThreadPoolExecutor.afterExecute(HadoopThreadPoolExecut

In [4]:
######
# for sample data
######
spark = SparkSession.builder.appName("DecisionTreeJob").getOrCreate()

csv_rdd1 = spark.sparkContext.textFile("gs://final-bucket-jy/balanced_flight_encoded.csv/part-00121-ef9393a8-06b1-4cd8-8720-6fb2f864254a-c000.csv")
csv_rdd2 = spark.sparkContext.textFile("gs://final-bucket-jy/balanced_flight_encoded.csv/part-00321-ef9393a8-06b1-4cd8-8720-6fb2f864254a-c000.csv")

# Convert the RDD1 to a DataFrame
csv_rows1 = csv_rdd1.map(lambda line: line.split(","))
header = csv_rows1.first()  # Extract header
csv_data1 = csv_rows1.filter(lambda row: row != header).map(lambda row: Row(**{header[i]: float(row[i]) for i in range(len(header))}))

# Convert the RDD2 to a DataFrame
csv_rows2 = csv_rdd2.map(lambda line: line.split(","))
csv_data2 = csv_rows2.filter(lambda row: row != header).map(lambda row: Row(**{header[i]: float(row[i]) for i in range(len(header))}))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/21 10:28:18 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
24/04/21 10:28:18 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
24/04/21 10:28:18 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
24/04/21 10:28:18 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
24/04/21 10:28:28 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #1,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at com.google.common.util.concurrent.FluentFuture$TrustedFuture.get(FluentFuture.java:88)
	at org.apache.hadoop.util.concurrent.ExecutorHelper.logThrowableFromAfterExecute(ExecutorHelper.java:48)
	at org.apache.hadoop.util.concurrent.HadoopThreadPoolExecutor.afterExecute(HadoopThreadPoolExecut

In [5]:
# Create a DataFrame from the RDD of Rows
sampled_df = spark.createDataFrame(csv_data)

# Limit the number of rows to 100
#sampled_df = sampled_df.limit(50)

In [38]:
sampled_df1 = spark.createDataFrame(csv_data1)
sampled_df2 = spark.createDataFrame(csv_data2)

In [39]:
# # Limit the number of rows to 100
# sampled_df1 = sampled_df1.limit(20000)
# sampled_df2 = sampled_df2.limit(20000)
sampled_df = sampled_df1.union(sampled_df2)
feature_df = sampled_df.drop('Delay')

In [7]:
sampled_df.count()

24/04/15 05:03:57 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


53316

In [6]:
counts = sampled_df.groupBy('Delay').count()
counts.show()

24/04/21 09:34:07 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----+-----+
|Delay|count|
+-----+-----+
|  0.0|26657|
|  1.0|26659|
+-----+-----+



In [16]:
counts1 = sampled_df1.groupBy('Delay').count()
counts1.show()

+-----+-----+
|Delay|count|
+-----+-----+
|  0.0|  500|
+-----+-----+



In [17]:
counts2 = sampled_df2.groupBy('Delay').count()
counts2.show()

+-----+-----+
|Delay|count|
+-----+-----+
|  1.0|  500|
+-----+-----+



In [21]:
sampled_df = sampled_df1.union(sampled_df2)

In [24]:
counts = sampled_df.groupBy('Delay').count()
counts.show()

+-----+-----+
|Delay|count|
+-----+-----+
|  0.0|   60|
|  1.0|   60|
+-----+-----+



In [13]:
# # Import the data as an RDD
# csv_rdd1 = spark.sparkContext.textFile("gs://final-bucket-jy/balanced_flight_encoded.csv/part-00285-1a134fbf-5193-4064-89a5-cb01a8fb1d12-c000.csv")

# csv_rows1 = csv_rdd1.map(lambda line: line.split(","))
# header = csv_rows1.first()  # Extract header
# csv_data1 = csv_rows1.filter(lambda row: row != header).map(lambda row: Row(**{header[i]: float(row[i]) for i in range(len(header))}))

In [14]:
# # Create a DataFrame from the RDD of Rows
# sampled_df1 = spark.createDataFrame(csv_data)

# counts1 = sampled_df1.groupBy('Delay').count()
# counts1.show()

+-----+-----+
|Delay|count|
+-----+-----+
|  0.0| 1298|
+-----+-----+



In [18]:
sampled_df.columns.index('Delay')

27

In [22]:
feature_df = sampled_df.drop('Delay')

In [8]:
feature_df.columns.index('Delay')

ValueError: 'Delay' is not in list

In [9]:
len(sampled_df.columns)

112

# Modeling

In [40]:
# Define feature columns
feature_columns = feature_df.columns  # Assuming the last column is the target

# Assemble features into a single vector column
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
#sampled_df_model = assembler.transform(sampled_df)
# Define a simple pipeline
pipeline = Pipeline(stages=[assembler])

# Fit the pipeline to the data
pipelineModel = pipeline.fit(sampled_df)

# Transform the data with the pipeline
sampled_df_transformed = pipelineModel.transform(sampled_df)
# Select features and target column for modeling
sampled_df_model = sampled_df_transformed.select("features", target_column)

# Split data into training and test sets
train_df, test_df = sampled_df_model.randomSplit([train_split, test_split])

In [9]:
###### Record accuracy ######
records = {} # the item we will collect "model_name": train_accuracy, train_auc, test_accuracy, test_auc

In [72]:
c = train_df.groupBy('Delay').count()
c.show()

+-----+-----+
|Delay|count|
+-----+-----+
|  0.0| 1145|
|  1.0| 1611|
+-----+-----+



In [73]:
c = test_df.groupBy('Delay').count()
c.show()

+-----+-----+
|Delay|count|
+-----+-----+
|  0.0|  264|
|  1.0|  396|
+-----+-----+



## 1. Logistic Regression

In [43]:
# Train a logistic regression model on the subset of data
lr = LogisticRegression(featuresCol='features', labelCol=target_column)
#lr_Model = lr.fit(train_df)

In [26]:
# Evaluate the model on the training set by accuracy
train_predictions = lr_Model.transform(train_df)
# by accuracy
evaluator_acc = MulticlassClassificationEvaluator(labelCol=target_column, predictionCol="prediction", metricName="accuracy")

# by auc
evaluator_auc = BinaryClassificationEvaluator(labelCol=target_column, rawPredictionCol="rawPrediction")
evaluator_auc.setMetricName("areaUnderROC")

accuracy = evaluator_acc.evaluate(train_predictions)
print(f"Training Accuracy: {accuracy}")

auc = evaluator_auc.evaluate(train_predictions, {evaluator_auc.metricName: "areaUnderROC"})
print(f"Area Under ROC: {auc}")

# Evaluate the model on the test set by accuracy
predictions = lr_Model.transform(test_df)
# by accuracy
accuracy_test = evaluator_acc.evaluate(predictions)
print(f"Test Accuracy: {accuracy_test}")

# Calculate the area under the ROC curve (AUC)
auc_test = evaluator_auc.evaluate(predictions, {evaluator_auc.metricName: "areaUnderROC"})
print(f"Area Under ROC: {auc_test}")

Training Accuracy: 0.7947810184751009


Area Under ROC: 0.866910050403447


Test Accuracy: 0.7871342835708928


Area Under ROC: 0.8597017031080224


In [27]:
records['logistic regression'] = [accuracy, auc, accuracy_test, auc_test]

NameError: name 'records' is not defined

## 2. Decision Tree

In [49]:
###### Modeling ######
# Initialize the DecisionTreeClassifier
dt = DecisionTreeClassifier(labelCol=target_column, featuresCol="features")

# Train the decision tree model
#dt_model = dt.fit(train_df)

In [29]:
# Evaluate the model on the training set by accuracy
train_predictions = dt_model.transform(train_df)

# by accuracy
evaluator_acc = MulticlassClassificationEvaluator(labelCol=target_column, predictionCol="prediction", metricName="accuracy")

# by auc
evaluator_auc = BinaryClassificationEvaluator(labelCol=target_column, rawPredictionCol="rawPrediction")
evaluator_auc.setMetricName("areaUnderROC")

accuracy = evaluator_acc.evaluate(train_predictions)
print(f"Training Accuracy: {accuracy}")
auc = evaluator_auc.evaluate(train_predictions, {evaluator_auc.metricName: "areaUnderROC"})
print(f"Area Under ROC: {auc}")

# Evaluate the model on the test set by accuracy
predictions = dt_model.transform(test_df)
# by accuracy
accuracy_test = evaluator_acc.evaluate(predictions)
print(f"Test Accuracy: {accuracy_test}")

# Calculate the area under the ROC curve (AUC)
auc_test = evaluator_auc.evaluate(predictions, {evaluator_auc.metricName: "areaUnderROC"})
print(f"Area Under ROC: {auc_test}")

Training Accuracy: 0.7772202944762262


Area Under ROC: 0.8120758443534942


Test Accuracy: 0.77344336084021


Area Under ROC: 0.8069675488377694


In [21]:
records['decision tree'] = [accuracy, auc, accuracy_test, auc_test]

## 3. Random Forest Classifier

In [54]:
###### Modeling ######
# Initialize the RandomForestClassifier
rf = RandomForestClassifier(labelCol=target_column, featuresCol="features")

# Train the decision tree model
#rf_model = rf.fit(train_df)

In [31]:
# Evaluate the model on the training set by accuracy
train_predictions = rf_model.transform(train_df)
evaluator_acc = MulticlassClassificationEvaluator(labelCol=target_column, predictionCol="prediction", metricName="accuracy")
evaluator_auc = BinaryClassificationEvaluator(labelCol=target_column, rawPredictionCol="rawPrediction", metricName="areaUnderROC")

train_accuracy = evaluator_acc.evaluate(train_predictions)
print(f"Training Accuracy: {train_accuracy}")
train_auc = evaluator_auc.evaluate(train_predictions)
print(f"Area Under ROC: {train_auc}")

# Evaluate the model on the test set by accuracy
test_predictions = rf_model.transform(test_df)
test_accuracy = evaluator_acc.evaluate(test_predictions)
print(f"Test Accuracy: {test_accuracy}")

# Calculate the area under the ROC curve (AUC)

auc = evaluator_auc.evaluate(test_predictions)
print(f"Area Under ROC: {auc}")

Training Accuracy: 0.7931867204351496


Area Under ROC: 0.8590832097730543


Test Accuracy: 0.7823518379594899


Area Under ROC: 0.8475931350322239


In [24]:
records['random forest classifier'] = [train_accuracy, train_auc, test_accuracy, auc]

## 4. Gradient Boosted Decision Trees

In [63]:
# Initialize Gradient Boosted Decision Trees Classifier
gbt = GBTClassifier(labelCol=target_column, featuresCol="features")

# Train the model
#gbt_model = gbt.fit(train_df)

In [33]:
# Evaluate the model on the training set by accuracy
train_predictions = gbt_model.transform(train_df)
evaluator_acc = MulticlassClassificationEvaluator(labelCol=target_column, predictionCol="prediction", metricName="accuracy")
evaluator_auc = BinaryClassificationEvaluator(labelCol=target_column, rawPredictionCol="rawPrediction", metricName="areaUnderROC")

train_accuracy = evaluator_acc.evaluate(train_predictions)
print(f"Training Accuracy: {train_accuracy}")
train_auc = evaluator_auc.evaluate(train_predictions)
print(f"Area Under ROC: {train_auc}")

# Evaluate the model on the test set by accuracy
test_predictions = gbt_model.transform(test_df)
test_accuracy = evaluator_acc.evaluate(test_predictions)
print(f"Test Accuracy: {test_accuracy}")

# Calculate the area under the ROC curve (AUC)

auc = evaluator_auc.evaluate(test_predictions)
print(f"Area Under ROC: {auc}")

24/04/21 02:18:12 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1712542704209_0040_01_000005 on host: final-cluster-jy-m.us-central1-c.c.final-project-419701.internal. Exit status: 143. Diagnostics: [2024-04-21 02:18:12.517]Container killed on request. Exit code is 143
[2024-04-21 02:18:12.517]Container exited with a non-zero exit code 143. 
[2024-04-21 02:18:12.519]Killed by external signal
.
24/04/21 02:18:12 ERROR org.apache.spark.scheduler.cluster.YarnScheduler: Lost executor 5 on final-cluster-jy-m.us-central1-c.c.final-project-419701.internal: Container from a bad node: container_1712542704209_0040_01_000005 on host: final-cluster-jy-m.us-central1-c.c.final-project-419701.internal. Exit status: 143. Diagnostics: [2024-04-21 02:18:12.517]Container killed on request. Exit code is 143
[2024-04-21 02:18:12.517]Container exited with a non-zero exit code 143. 
[2024-04-21 02:18:12.519]Killed by external signal
.
24/04/21 02:18:12 WARN org.apache.

Training Accuracy: 0.8662196380005627


Area Under ROC: 0.9529277662583844


Test Accuracy: 0.8555888972243061


Area Under ROC: 0.9480334625867375


In [11]:
records['gradient boosted decision tree'] = [train_accuracy, train_auc, test_accuracy, auc]

## 5. (Run on smaller data) Multilayer perceptron (MLP) 

In [11]:
layers = [111, 64, 32, 2]

mlp = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, labelCol=target_column, featuresCol="features")

mlp_model = mlp.fit(train_df)
result = mlp_model.transform(train_df)#mlp_model.transform(test_df)

evaluator = MulticlassClassificationEvaluator(labelCol=target_column,metricName="accuracy")
evaluator_auc = BinaryClassificationEvaluator(labelCol=target_column, rawPredictionCol="rawPrediction", metricName="areaUnderROC")

accuracy = evaluator.evaluate(result)
print("Test set accuracy = " + str(accuracy))

# Evaluate the model on the test set by accuracy
test_predictions = mlp_model.transform(test_df)
test_accuracy = evaluator.evaluate(test_predictions)
print(f"Test Accuracy: {test_accuracy}")

# Calculate the area under the ROC curve (AUC)

auc = evaluator_auc.evaluate(test_predictions)
print(f"Area Under ROC: {auc}")

24/04/21 10:13:26 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/04/21 10:13:26 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


Test set accuracy = 0.5116279069767442
Test Accuracy: 0.7142857142857143
Area Under ROC: 0.75


In [12]:
# define layers for the neural network
# [input layer of size 114, two intermdeidate of size 64 and 32, output size 2]
layers = [10, 2,2]

# Create the trainer and set its parameters
mlp = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234, labelCol=target_column, featuresCol="features")

# Train the model
mlp_model = mlp.fit(train_df)

24/04/21 05:37:38 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/04/21 05:37:38 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [41]:
# define layers for the neural network
# [input layer of size 114, two intermdeidate of size 64 and 32, output size 2]
layers = [111, 64, 32, 2]

# Create the trainer and set its parameters
mlp = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234, labelCol=target_column, featuresCol="features")

# Train the model
mlp_model = mlp.fit(train_df)

In [42]:
# Evaluate the model on the training set by accuracy
train_predictions = mlp_model.transform(train_df)
evaluator_acc = MulticlassClassificationEvaluator(labelCol=target_column, predictionCol="prediction", metricName="accuracy")
evaluator_auc = BinaryClassificationEvaluator(labelCol=target_column, rawPredictionCol="rawPrediction", metricName="areaUnderROC")

train_accuracy = evaluator_acc.evaluate(train_predictions)
print(f"Training Accuracy: {train_accuracy}")
train_auc = evaluator_auc.evaluate(train_predictions)
print(f"Area Under ROC: {train_auc}")

# Evaluate the model on the test set by accuracy
test_predictions = mlp_model.transform(test_df)
test_accuracy = evaluator_acc.evaluate(test_predictions)
print(f"Test Accuracy: {test_accuracy}")

# Calculate the area under the ROC curve (AUC)

auc = evaluator_auc.evaluate(test_predictions)
print(f"Area Under ROC: {auc}")

Training Accuracy: 0.5849056603773585
Area Under ROC: 0.5022557797240044
Test Accuracy: 0.6
Area Under ROC: 0.5031756963575145


In [ ]:
records['multilayer perceptron'] = [train_accuracy, train_auc, test_accuracy, auc]

## 6. Linear Support Vector Machine

In [74]:
# Initialize Linear Support Vector Machine
lsvc = LinearSVC(maxIter=10, regParam=0.1, labelCol=target_column, featuresCol="features")

# Train the model
#lsvc_model = lsvc.fit(train_df)

In [12]:
# Evaluate the model on the training set by accuracy
train_predictions = lsvc_model.transform(train_df)
evaluator_acc = MulticlassClassificationEvaluator(labelCol=target_column, predictionCol="prediction", metricName="accuracy")
evaluator_auc = BinaryClassificationEvaluator(labelCol=target_column, rawPredictionCol="rawPrediction", metricName="areaUnderROC")

train_accuracy = evaluator_acc.evaluate(train_predictions)
print(f"Training Accuracy: {train_accuracy}")
train_auc = evaluator_auc.evaluate(train_predictions)
print(f"Area Under ROC: {train_auc}")

# Evaluate the model on the test set by accuracy
test_predictions = lsvc_model.transform(test_df)
test_accuracy = evaluator_acc.evaluate(test_predictions)
print(f"Test Accuracy: {test_accuracy}")

# Calculate the area under the ROC curve (AUC)

auc = evaluator_auc.evaluate(test_predictions)
print(f"Area Under ROC: {auc}")

Training Accuracy: 0.7547522301620663


Area Under ROC: 0.831187526181334


Test Accuracy: 0.752334667429007


24/04/21 02:29:27 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1712542704209_0041_01_000002 on host: final-cluster-jy-m.us-central1-c.c.final-project-419701.internal. Exit status: 143. Diagnostics: [2024-04-21 02:29:27.205]Container killed on request. Exit code is 143
[2024-04-21 02:29:27.205]Container exited with a non-zero exit code 143. 
[2024-04-21 02:29:27.205]Killed by external signal
.
24/04/21 02:29:27 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 2 for reason Container from a bad node: container_1712542704209_0041_01_000002 on host: final-cluster-jy-m.us-central1-c.c.final-project-419701.internal. Exit status: 143. Diagnostics: [2024-04-21 02:29:27.205]Container killed on request. Exit code is 143
[2024-04-21 02:29:27.205]Container exited with a non-zero exit code 143. 
[2024-04-21 02:29:27.205]Killed by external signal
.
24/04/21 02:29:27 ERROR org.apache.spark.

Area Under ROC: 0.8268426801325668


In [15]:
records['linear support vector machine'] = [train_accuracy, train_auc, test_accuracy, auc]

## 7. One-vs-Rest (OneVsRest)

is a strategy commonly used in multi-class classification problems. In this strategy, you train multiple binary classifiers, each specializing in distinguishing one class from all other classes.

In [68]:
# Initialize one-vs-rest model
ovr = OneVsRest(classifier=lr, labelCol=target_column, featuresCol="features")

# Train the model
#ovr_model = ovr.fit(train_df)

In [16]:
# Evaluate the model on the training set by accuracy
train_predictions = ovr_model.transform(train_df)
evaluator_acc = MulticlassClassificationEvaluator(labelCol=target_column, predictionCol="prediction", metricName="accuracy")
evaluator_auc = BinaryClassificationEvaluator(labelCol=target_column, rawPredictionCol="rawPrediction", metricName="areaUnderROC")

train_accuracy = evaluator_acc.evaluate(train_predictions)
print(f"Training Accuracy: {train_accuracy}")
train_auc = evaluator_auc.evaluate(train_predictions)
print(f"Area Under ROC: {train_auc}")

# Evaluate the model on the test set by accuracy
test_predictions = ovr_model.transform(test_df)
test_accuracy = evaluator_acc.evaluate(test_predictions)
print(f"Test Accuracy: {test_accuracy}")

# Calculate the area under the ROC curve (AUC)

auc = evaluator_auc.evaluate(test_predictions)
print(f"Area Under ROC: {auc}")

24/04/21 02:34:51 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1712542704209_0041_01_000004 on host: final-cluster-jy-m.us-central1-c.c.final-project-419701.internal. Exit status: 143. Diagnostics: [2024-04-21 02:34:51.153]Container killed on request. Exit code is 143
[2024-04-21 02:34:51.153]Container exited with a non-zero exit code 143. 
[2024-04-21 02:34:51.153]Killed by external signal
.
24/04/21 02:34:51 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 4 for reason Container from a bad node: container_1712542704209_0041_01_000004 on host: final-cluster-jy-m.us-central1-c.c.final-project-419701.internal. Exit status: 143. Diagnostics: [2024-04-21 02:34:51.153]Container killed on request. Exit code is 143
[2024-04-21 02:34:51.153]Container exited with a non-zero exit code 143. 
[2024-04-21 02:34:51.153]Killed by external signal
.
24/04/21 02:34:51 ERROR org.apache.spark.

Training Accuracy: 0.796179533884452


Area Under ROC: 0.8669025243425479


24/04/21 02:35:38 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1712542704209_0041_01_000005 on host: final-cluster-jy-m.us-central1-c.c.final-project-419701.internal. Exit status: 143. Diagnostics: [2024-04-21 02:35:38.173]Container killed on request. Exit code is 143
[2024-04-21 02:35:38.174]Container exited with a non-zero exit code 143. 
[2024-04-21 02:35:38.175]Killed by external signal
.
24/04/21 02:35:38 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 5 for reason Container from a bad node: container_1712542704209_0041_01_000005 on host: final-cluster-jy-m.us-central1-c.c.final-project-419701.internal. Exit status: 143. Diagnostics: [2024-04-21 02:35:38.173]Container killed on request. Exit code is 143
[2024-04-21 02:35:38.174]Container exited with a non-zero exit code 143. 
[2024-04-21 02:35:38.175]Killed by external signal
.
24/04/21 02:35:38 ERROR org.apache.spark.

Test Accuracy: 0.7887364208118925


Area Under ROC: 0.8597131231502189


In [23]:
records['linear support vector machine'] = [train_accuracy, train_auc, test_accuracy, auc]

## 8. AutoML

In [20]:
pandas_df = sampled_df.toPandas()

h2o.init()

h2o_df = h2o.H2OFrame(pandas_df)

# Split the data into train and test sets
train, test = h2o_df.split_frame(ratios=[.8], seed=1234)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_402"; OpenJDK Runtime Environment (Temurin)(build 1.8.0_402-b06); OpenJDK 64-Bit Server VM (Temurin)(build 25.402-b06, mixed mode)
  Starting server from /opt/conda/miniconda3/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp5nizx8fe
  JVM stdout: /tmp/tmp5nizx8fe/h2o_root_started_from_python.out
  JVM stderr: /tmp/tmp5nizx8fe/h2o_root_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,1 month and 7 days
H2O_cluster_name:,H2O_from_python_root_tcvc8m
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.474 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [23]:
# Identify predictors and response
x = train.columns
y = "Delay"
x.remove(y)

In [ ]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML
aml = H2OAutoML(max_runtime_secs=3600, max_models=12, seed=1)
aml.train(x=x, y=y, training_frame=train)

# View the AutoML Leaderboard
lb = aml.leaderboard
print(lb.head(rows=lb.nrows))  

AutoML progress: |
02:40:10.189: _train param, Dropping bad and constant columns: [arrival_airport_min_altitude, arrival_airport_min_knots, bit_0, bit_1, departure_airport_avg_altitude, departure_airport_max_altitude, arrival_airport_max_altitude, departure_airport_max_bearing, bit_4, arrival_airport_max_bearing, bit_2, bit_3, departure_airport_min_altitude, arrival_airport_avg_altitude, departure_airport_min_knots]

██████
02:44:16.194: _train param, Dropping bad and constant columns: [arrival_airport_min_altitude, arrival_airport_min_knots, bit_0, bit_1, departure_airport_avg_altitude, departure_airport_max_altitude, arrival_airport_max_altitude, departure_airport_max_bearing, bit_4, arrival_airport_max_bearing, bit_2, bit_3, departure_airport_min_altitude, arrival_airport_avg_altitude, departure_airport_min_knots]


02:44:58.556: _train param, Dropping bad and constant columns: [arrival_airport_min_altitude, arrival_airport_min_knots, bit_0, bit_1, departure_airport_avg_altitude, de

In [27]:
# The leader model is the best model
leader_model = aml.leader
predictions = leader_model.predict(test)
performance = leader_model.model_performance(test)
print(performance)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.008079970190256794
RMSE: 0.0898886544023037
LogLoss: 0.03344070475011655
Mean Per-Class Error: 0.00991751316689991
AUC: 0.9987448560323741
AUCPR: 0.9989567940452997
Gini: 0.9974897120647481

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.506258468100948
       0     1     Error    Rate
-----  ----  ----  -------  ---------------
0      5180  43    0.0082   (43.0/5223.0)
1      63    5367  0.0116   (63.0/5430.0)
Total  5243  5410  0.01     (106.0/10653.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.506258     0.990221  189
max f2                       0.241969     0.991909  238
max f0point5                 0.738359     0.992677  155
max accuracy                

# (Run on smaller data) Hyperparameter Tuning

## 1. Cross Validation

In [11]:
# collect accuarcy and auc for cross validataion
cv_records = {}

### 1.1 Logistic Regression

In [44]:
# Define parameter grid
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 1.0]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

In [45]:
# Define evaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="Delay")

# Set up 5-fold cross-validation
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)


# Run cross-validation, and choose the best set of parameters
cvModel = crossval.fit(train_df)

24/04/21 10:39:00 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/04/21 10:40:05 ERROR breeze.optimize.LBFGS: Failure! Resetting history: breeze.optimize.FirstOrderException: Line search failed


In [46]:
# Fetch the best model
bestModel = cvModel.bestModel

# Make predictions on the test set
train_predictions = bestModel.transform(train_df)
test_predictions = bestModel.transform(test_df)

In [47]:
# Evaluate the best model's performance
accuracy_train = train_predictions.filter(train_predictions['Delay'] == train_predictions['prediction']).count() / train_predictions.count()
accuracy_test = test_predictions.filter(test_predictions['Delay'] == test_predictions['prediction']).count() / test_predictions.count()
print(f"Best Model Accuracy: {accuracy_train}")
print(f"Best Model Accuracy: {accuracy_test}")

# Calculate the area under the ROC curve (AUC)
auc_train = evaluator.evaluate(train_predictions, {evaluator.metricName: "areaUnderROC"})
auc_test = evaluator.evaluate(test_predictions, {evaluator.metricName: "areaUnderROC"})
print(f"Area Under ROC: {auc_train}")
print(f"Area Under ROC: {auc_test}")

Best Model Accuracy: 0.9296081277213353
Best Model Accuracy: 0.9303030303030303
Area Under ROC: 0.9362521312266379
Area Under ROC: 0.9368686868686869


In [48]:
# view the best model's parameters
best_regParam = bestModel._java_obj.getRegParam()
best_elasticNetParam = bestModel._java_obj.getElasticNetParam()
print(f"Best Reg Param: {best_regParam}, \nBest Elastic Net Param: {best_elasticNetParam}")

Best Reg Param: 0.01, 
Best Elastic Net Param: 1.0


In [ ]:
cv_records['logistic regression'] = [accuracy_train, auc_train, accuracy_test, auc_test]

### 1.2 Decision Tree

In [50]:
# Define a grid of hyperparameters to search
param_grid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [5, 10, 15]) \
    .addGrid(dt.maxBins, [20, 30, 40]) \
    .build()

# Set up the cross-validation
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="Delay")
crossval = CrossValidator(estimator=dt,
                          estimatorParamMaps=param_grid,
                          evaluator=evaluator,
                          numFolds=5)

# Run cross-validation to find the best hyperparameters
cv_model = crossval.fit(train_df)

In [51]:
# Fetch the best model
best_model = cv_model.bestModel

# Make predictions on the train,test set
train_predictions = best_model.transform(train_df)
test_predictions = best_model.transform(test_df)

In [52]:
# Evaluate the best model's performance
accuracy_train = train_predictions.filter(train_predictions['Delay'] == train_predictions['prediction']).count() / train_predictions.count()
accuracy_test = test_predictions.filter(test_predictions['Delay'] == test_predictions['prediction']).count() / test_predictions.count()
print(f"Best Model Accuracy: {accuracy_train}")
print(f"Best Model Accuracy: {accuracy_test}")

# Calculate the area under the ROC curve (AUC)
auc_train = evaluator.evaluate(train_predictions, {evaluator.metricName: "areaUnderROC"})
auc_test = evaluator.evaluate(test_predictions, {evaluator.metricName: "areaUnderROC"})
print(f"Area Under ROC: {auc_train}")
print(f"Area Under ROC: {auc_test}")

Best Model Accuracy: 0.9851233671988389
Best Model Accuracy: 0.9681818181818181
Area Under ROC: 0.9870223545005815
Area Under ROC: 0.970959595959596


In [53]:
# best model's parameters
best_maxDepth = best_model._java_obj.getMaxDepth()
best_maxBins = best_model._java_obj.getMaxBins()
print(f"Best Max Depth: {best_maxDepth}, Best Max Bins: {best_maxBins}")

Best Max Depth: 15, Best Max Bins: 30


In [45]:
cv_records['decision tree'] = [accuracy_train, auc_train, accuracy_test, auc_test]

### 1.3 Random Forest Classifier

#### without early stop

In [55]:
# Define a grid of hyperparameters to search
param_grid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20, 30]) \
    .addGrid(rf.maxDepth, [5, 10]) \
    .build()

# Set up the cross-validation
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="Delay")
cross_val = CrossValidator(estimator=rf,
                           estimatorParamMaps=param_grid,
                           evaluator=evaluator,
                           numFolds=5)

# Run cross-validation to find the best hyperparameters
cv_model = cross_val.fit(train_df)

In [56]:
# Fetch the best model
best_model = cv_model.bestModel

# Make predictions on the train,test set
train_predictions = best_model.transform(train_df)
test_predictions = best_model.transform(test_df)

In [57]:
# Evaluate the best model's performance
accuracy_train = train_predictions.filter(train_predictions['Delay'] == train_predictions['prediction']).count() / train_predictions.count()
accuracy_test = test_predictions.filter(test_predictions['Delay'] == test_predictions['prediction']).count() / test_predictions.count()
print(f"Best Model Accuracy: {accuracy_train}")
print(f"Best Model Accuracy: {accuracy_test}")

# Calculate the area under the ROC curve (AUC)
auc_train = evaluator.evaluate(train_predictions, {evaluator.metricName: "areaUnderROC"})
auc_test = evaluator.evaluate(test_predictions, {evaluator.metricName: "areaUnderROC"})
print(f"Area Under ROC: {auc_train}")
print(f"Area Under ROC: {auc_test}")

Best Model Accuracy: 0.9796806966618288
Best Model Accuracy: 0.9666666666666667
Area Under ROC: 0.9824931760088259
Area Under ROC: 0.9690656565656567


In [58]:
# best model's parameters
best_numTrees = best_model._java_obj.getNumTrees()
best_maxDepth = best_model._java_obj.getMaxDepth()
print(f"Best Num Trees: {best_numTrees}, Best Max Depth: {best_maxDepth}")

Best Num Trees: 20, Best Max Depth: 10


In [50]:
cv_records['random forest classifier'] = [accuracy_train, auc_train, accuracy_test, auc_test]

#### with early stop

In [59]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="Delay")
bestauc = float('-inf')
bestmodel = None
accuracy_list = []    # accuracy is the matrix that we want to use for early stopping
stop_tolerance = 0.01 # if accuracy does not imporve by at least 0.001, the training stops
stop_round = 5        # stop training if there is no improvement in accuracy for 10 consecutive rounds

condition_met = 0

for param in param_grid:
    num_trees, max_depth = param
    rf = RandomForestClassifier(numTrees=param[num_trees], maxDepth=param[max_depth], labelCol=target_column, featuresCol="features")
    model = rf.fit(train_df)

    predictions = model.transform(test_df)
    auc = evaluator.evaluate(predictions)

    if auc > bestauc:
        bestauc = auc
        bestmodel = model

    # Add early stopping condition here
    accuracy = predictions.filter(predictions['Delay'] == predictions['prediction']).count() / predictions.count()
    
    if accuracy_list:
        last_accuracy = accuracy_list[-1]
        if abs(accuracy-last_accuracy) < stop_tolerance:
            condition_met += 1
    accuracy_list.append(accuracy)

    if condition_met == stop_round:
        break

In [60]:
# Make predictions on the train,test set
train_predictions = bestmodel.transform(train_df)
test_predictions = bestmodel.transform(test_df)

In [61]:
# Evaluate the best model's performance
accuracy_train = train_predictions.filter(train_predictions['Delay'] == train_predictions['prediction']).count() / train_predictions.count()
accuracy_test = test_predictions.filter(test_predictions['Delay'] == test_predictions['prediction']).count() / test_predictions.count()
# Calculate the area under the ROC curve (AUC)
auc_train = evaluator.evaluate(train_predictions, {evaluator.metricName: "areaUnderROC"})
auc_test = evaluator.evaluate(test_predictions, {evaluator.metricName: "areaUnderROC"})

print("Training data:")
print(f"Best Model Accuracy: {accuracy_train}")
print(f"Area Under ROC: {auc_train}")
print("Test data:")
print(f"Best Model Accuracy: {accuracy_test}")
print(f"Area Under ROC: {auc_test}")

Training data:
Best Model Accuracy: 0.9840348330914369
Area Under ROC: 0.9858386258230126
Test data:
Best Model Accuracy: 0.9681818181818181
Area Under ROC: 0.9696969696969697


In [62]:
# best model's parameters
best_numTrees = bestmodel._java_obj.getNumTrees()
best_maxDepth = bestmodel._java_obj.getMaxDepth()
print(f"Best Num Trees: {best_numTrees} \nBest Max Depth: {best_maxDepth}")

Best Num Trees: 30 
Best Max Depth: 10


In [57]:
cv_records['random forest classifier (early stop)'] = [accuracy_train, auc_train, accuracy_test, auc_test]

### 1.4 Gradient Boosted Decision Tress

#### without early stop

In [ ]:
# Define a grid of hyperparameters to search
param_grid = ParamGridBuilder() \
    .addGrid(gbt.maxDepth, [3, 5, 10]) \
    .addGrid(gbt.maxIter, [5, 10, 20]) \
    .addGrid(gbt.stepSize, [0.1, 0.05]) \
    .build()

# Set up the cross-validation
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="Delay")
cross_val = CrossValidator(estimator=gbt,
                           estimatorParamMaps=param_grid,
                           evaluator=evaluator,
                           numFolds=5)

# Run cross-validation to find the best hyperparameters
cv_model = cross_val.fit(train_df)

In [ ]:
# Fetch the best model
best_model = cv_model.bestModel

# Make predictions on the train,test set
train_predictions = best_model.transform(train_df)
test_predictions = best_model.transform(test_df)

In [ ]:
# Evaluate the best model's performance
accuracy_train = train_predictions.filter(train_predictions['Delay'] == train_predictions['prediction']).count() / train_predictions.count()
accuracy_test = test_predictions.filter(test_predictions['Delay'] == test_predictions['prediction']).count() / test_predictions.count()
print(f"Best Model Accuracy: {accuracy_train}")
print(f"Best Model Accuracy: {accuracy_test}")

# Calculate the area under the ROC curve (AUC)
auc_train = evaluator.evaluate(train_predictions, {evaluator.metricName: "areaUnderROC"})
auc_test = evaluator.evaluate(test_predictions, {evaluator.metricName: "areaUnderROC"})
print(f"Area Under ROC: {auc_train}")
print(f"Area Under ROC: {auc_test}")

Best Model Accuracy: 0.9992743105950653
Best Model Accuracy: 0.9757575757575757
Area Under ROC: 0.9993792675356921
Area Under ROC: 0.9766414141414143


In [ ]:
# best model's parameters
best_maxDepth = best_model._java_obj.getMaxDepth()
best_maxIter = best_model._java_obj.getMaxIter()
best_stepSize = best_model._java_obj.getStepSize()
print(f"Best Max Depth: {best_maxDepth}, Best Max Iter: {best_maxIter}, Best Step Size: {best_stepSize}")

Best Max Depth: 10, Best Max Iter: 20, Best Step Size: 0.1


In [62]:
cv_records['gradient boosted decision trees'] = [accuracy_train, auc_train, accuracy_test, auc_test]

#### with early stop

In [68]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="Delay")
bestauc = float('-inf')
bestmodel = None
accuracy_list = []    # accuracy is the matrix that we want to use for early stopping
stop_tolerance = 0.01 # if accuracy does not imporve by at least 0.001, the training stops
stop_round = 5        # stop training if there is no improvement in accuracy for 10 consecutive rounds

condition_met = 0

for param in param_grid:
    max_depth, max_iter, step_size = param
    gbt = GBTClassifier(maxDepth=param[max_depth], maxIter=param[max_iter], stepSize=param[step_size], labelCol=target_column, featuresCol="features")
    model = gbt.fit(train_df)

    predictions = model.transform(test_df)
    auc = evaluator.evaluate(predictions)

    if auc > bestauc:
        bestauc = auc
        bestmodel = model

    # Add early stopping condition here
    accuracy = predictions.filter(predictions['Delay'] == predictions['prediction']).count() / predictions.count()
    
    if accuracy_list:
        last_accuracy = accuracy_list[-1]
        if abs(accuracy-last_accuracy) < stop_tolerance:
            condition_met += 1
    accuracy_list.append(accuracy)

    if condition_met == stop_round:
        break

In [69]:
# Make predictions on the train,test set
train_predictions = bestmodel.transform(train_df)
test_predictions = bestmodel.transform(test_df)

In [70]:
# Evaluate the best model's performance
accuracy_train = train_predictions.filter(train_predictions['Delay'] == train_predictions['prediction']).count() / train_predictions.count()
accuracy_test = test_predictions.filter(test_predictions['Delay'] == test_predictions['prediction']).count() / test_predictions.count()
# Calculate the area under the ROC curve (AUC)
auc_train = evaluator.evaluate(train_predictions, {evaluator.metricName: "areaUnderROC"})
auc_test = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})

print("Training data:")
print(f"Best Model Accuracy: {accuracy_train}")
print(f"Area Under ROC: {auc_train}")
print("Test data:")
print(f"Best Model Accuracy: {accuracy_test}")
print(f"Area Under ROC: {auc_test}")

Training data:
Best Model Accuracy: 0.9909288824383164
Area Under ROC: 0.9922408441961514
Test data:
Best Model Accuracy: 0.9681818181818181
Area Under ROC: 0.9659090909090909


In [71]:
# best model's parameters
best_maxDepth = bestmodel._java_obj.getMaxDepth()
best_maxIter = bestmodel._java_obj.getMaxIter()
best_stepSize = bestmodel._java_obj.getStepSize()
print(f"Best Max Depth: {best_maxDepth} \nBest Max Iter: {best_maxIter} \nBest Step Size: {best_stepSize}")

Best Max Depth: 10 
Best Max Iter: 5 
Best Step Size: 0.1


In [67]:
cv_records['gradient boosted decision trees (early stop)'] = [accuracy_train, auc_train, accuracy_test, auc_test]

### 1.5 Multilayer Perceptron (MLP)

#### without early stop

In [83]:
param_grid = ParamGridBuilder() \
    .addGrid(mlp.maxIter, [100, 200]) \
    .addGrid(mlp.blockSize, [128, 256]) \
    .addGrid(mlp.layers, [[114, 5, 4, 2], [114, 6, 3, 2]]) \
    .build()

# Set up the cross-validation
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="Delay")
cross_val = CrossValidator(estimator=mlp,
                           estimatorParamMaps=param_grid,
                           evaluator=evaluator,
                           numFolds=5)

# Run cross-validation to find the best hyperparameters
cv_model = cross_val.fit(train_df)

24/04/21 11:08:27 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 1.0 in stage 30197.0 (TID 109115) (final-cluster-jy-m.us-central1-c.c.final-project-419701.internal executor 1): java.lang.ArrayIndexOutOfBoundsException
	at org.apache.spark.ml.ann.DataStacker.$anonfun$stack$4(Layer.scala:665)
	at org.apache.spark.ml.ann.DataStacker.$anonfun$stack$4$adapted(Layer.scala:664)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.ml.ann.DataStacker.$anonfun$stack$3(Layer.scala:664)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.storage.memory.MemoryStore.putIterator(MemoryStore.scala:222)
	at org.apache.spark.storage.memory.MemoryStore.putIteratorAsValues(MemoryStore.scala:299)
	at org.apache.spark.storage.BlockManager.$anonfun$doPutIterator$1(BlockManager.scala:1431)
	at org.apache.spark.storage.BlockManager.org$apache$spark$storage$BlockManager$$doPut

Py4JJavaError: An error occurred while calling o68794.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 30197.0 failed 4 times, most recent failure: Lost task 0.3 in stage 30197.0 (TID 109122) (final-cluster-jy-m.us-central1-c.c.final-project-419701.internal executor 1): java.lang.ArrayIndexOutOfBoundsException
	at org.apache.spark.ml.ann.DataStacker.$anonfun$stack$4(Layer.scala:665)
	at org.apache.spark.ml.ann.DataStacker.$anonfun$stack$4$adapted(Layer.scala:664)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.ml.ann.DataStacker.$anonfun$stack$3(Layer.scala:664)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.storage.memory.MemoryStore.putIterator(MemoryStore.scala:222)
	at org.apache.spark.storage.memory.MemoryStore.putIteratorAsValues(MemoryStore.scala:299)
	at org.apache.spark.storage.BlockManager.$anonfun$doPu

In [84]:
# Fetch the best model
best_model = cv_model.bestModel

# Make predictions on the train,test set
train_predictions = best_model.transform(train_df)
test_predictions = best_model.transform(test_df)

In [85]:
# Evaluate the best model's performance
accuracy_train = train_predictions.filter(train_predictions['Delay'] == train_predictions['prediction']).count() / train_predictions.count()
accuracy_test = test_predictions.filter(test_predictions['Delay'] == test_predictions['prediction']).count() / test_predictions.count()
print(f"Best Model Accuracy: {accuracy_train}")
print(f"Best Model Accuracy: {accuracy_test}")

# Calculate the area under the ROC curve (AUC)
auc_train = evaluator.evaluate(train_predictions, {evaluator.metricName: "areaUnderROC"})
auc_test = evaluator.evaluate(test_predictions, {evaluator.metricName: "areaUnderROC"})
print(f"Area Under ROC: {auc_train}")
print(f"Area Under ROC: {auc_test}")

Best Model Accuracy: 0.8439767779390421
Best Model Accuracy: 0.8454545454545455
Area Under ROC: 0.8314269527999372
Area Under ROC: 0.8295454545454546


In [86]:
# best model's parameters
best_maxIter = best_model._java_obj.getMaxIter()
best_blockSize = best_model._java_obj.getBlockSize()
best_layers = best_model._java_obj.getLayers()
print(f"Best Max Iter: {best_maxIter} \nBest Block Size: {best_blockSize} \nBest Layers: {best_layers}")

Py4JError: An error occurred while calling o67874.getBlockSize. Trace:
py4j.Py4JException: Method getBlockSize([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:750)



In [72]:
cv_records['multilayer perceptron'] = [accuracy_train, auc_train, accuracy_test, auc_test]

#### with early stop

In [74]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="Delay")
bestauc = float('-inf')
bestmodel = None
accuracy_list = []    # accuracy is the matrix that we want to use for early stopping
stop_tolerance = 0.01 # if accuracy does not imporve by at least 0.001, the training stops
stop_round = 5        # stop training if there is no improvement in accuracy for 10 consecutive rounds

condition_met = 0

for param in param_grid:
    max_iter, block_size, layer = param
    mlp = MultilayerPerceptronClassifier(maxIter=param[max_iter], blockSize=param[block_size], layers=param[layer], labelCol=target_column, featuresCol="features")
    model = mlp.fit(train_df)

    predictions = model.transform(test_df)
    auc = evaluator.evaluate(predictions)

    if auc > bestauc:
        bestauc = auc
        bestmodel = model

    # Add early stopping condition
    accuracy = predictions.filter(predictions['Delay'] == predictions['prediction']).count() / predictions.count()
    
    if accuracy_list:
        last_accuracy = accuracy_list[-1]
        if abs(accuracy-last_accuracy) < stop_tolerance:
            condition_met += 1
    accuracy_list.append(accuracy)

    if condition_met == stop_round:
        break

TypeError: Invalid param value given for param "layers". Could not convert 0.1 to list of ints

In [ ]:
# Make predictions on the train,test set
train_predictions = bestmodel.transform(train_df)
test_predictions = bestmodel.transform(test_df)

In [ ]:
# Evaluate the best model's performance
accuracy_train = train_predictions.filter(train_predictions['Delay'] == train_predictions['prediction']).count() / train_predictions.count()
accuracy_test = test_predictions.filter(test_predictions['Delay'] == test_predictions['prediction']).count() / test_predictions.count()
# Calculate the area under the ROC curve (AUC)
auc_train = evaluator.evaluate(train_predictions, {evaluator.metricName: "areaUnderROC"})
auc_test = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})

print("Training data:")
print(f"Best Model Accuracy: {accuracy_train}")
print(f"Area Under ROC: {auc_train}")
print("Test data:")
print(f"Best Model Accuracy: {accuracy_test}")
print(f"Area Under ROC: {auc_test}")

In [ ]:
# best model's parameters
best_maxIter = bestmodel._java_obj.getMaxIter()
best_blockSize = bestmodel._java_obj.getBlockSize()
best_layers = bestmodel._java_obj.getLayers()
print(f"Best Max Iter: {best_maxIter} \nBest Block Size: {best_blockSize} \nBest Layers: {best_layers}")

In [ ]:
cv_records['multilayer perceptron (early stop)'] = [accuracy_train, auc_train, accuracy_test, auc_test]

### 1.6 Linear Support Vector Machine

#### without early stop

In [75]:
param_grid = ParamGridBuilder() \
    .addGrid(lsvc.regParam, [0.1, 0.01]) \
    .addGrid(lsvc.maxIter, [10, 20]) \
    .build()

# Set up the cross-validation
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="Delay")
cross_val = CrossValidator(estimator=lsvc,
                           estimatorParamMaps=param_grid,
                           evaluator=evaluator,
                           numFolds=5)

# Run cross-validation to find the best hyperparameters
cv_model = cross_val.fit(train_df)

In [76]:
# Fetch the best model
best_model = cv_model.bestModel

# Make predictions on the train,test set
train_predictions = best_model.transform(train_df)
test_predictions = best_model.transform(test_df)

In [77]:
# Evaluate the best model's performance
accuracy_train = train_predictions.filter(train_predictions['Delay'] == train_predictions['prediction']).count() / train_predictions.count()
accuracy_test = test_predictions.filter(test_predictions['Delay'] == test_predictions['prediction']).count() / test_predictions.count()
print(f"Best Model Accuracy: {accuracy_train}")
print(f"Best Model Accuracy: {accuracy_test}")

# Calculate the area under the ROC curve (AUC)
auc_train = evaluator.evaluate(train_predictions, {evaluator.metricName: "areaUnderROC"})
auc_test = evaluator.evaluate(test_predictions, {evaluator.metricName: "areaUnderROC"})
print(f"Area Under ROC: {auc_train}")
print(f"Area Under ROC: {auc_test}")

Best Model Accuracy: 0.8439767779390421
Best Model Accuracy: 0.8454545454545455
Area Under ROC: 0.8314269527999372
Area Under ROC: 0.8295454545454546


In [78]:
# best model's parameters
best_regParam = best_model._java_obj.getRegParam()
best_maxIter = best_model._java_obj.getMaxIter()
print(f"Best Reg Param: {best_regParam} \nBest Max Iter: {best_maxIter}")

Best Reg Param: 0.01 
Best Max Iter: 20


In [81]:
cv_records['linear support vector machine'] = [accuracy_train, auc_train, accuracy_test, auc_test]

#### with early stop

In [79]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="Delay")
bestauc = float('-inf')
bestmodel = None
accuracy_list = []    # accuracy is the matrix that we want to use for early stopping
stop_tolerance = 0.01 # if accuracy does not imporve by at least 0.001, the training stops
stop_round = 5        # stop training if there is no improvement in accuracy for 10 consecutive rounds

condition_met = 0

for param in param_grid:
    reg_param,max_iter = param
    lsvc = LinearSVC(maxIter=param[max_iter], regParam=param[reg_param], labelCol=target_column, featuresCol="features")
    model = lsvc.fit(train_df)

    predictions = model.transform(test_df)
    auc = evaluator.evaluate(predictions)

    if auc > bestauc:
        bestauc = auc
        bestmodel = model

    # Add early stopping condition here
    accuracy = predictions.filter(predictions['Delay'] == predictions['prediction']).count() / predictions.count()
    
    if accuracy_list:
        last_accuracy = accuracy_list[-1]
        if abs(accuracy-last_accuracy) < stop_tolerance:
            condition_met += 1
    accuracy_list.append(accuracy)

    if condition_met == stop_round:
        break

In [80]:
# Make predictions on the train,test set
train_predictions = bestmodel.transform(train_df)
test_predictions = bestmodel.transform(test_df)

In [81]:
# Evaluate the best model's performance
accuracy_train = train_predictions.filter(train_predictions['Delay'] == train_predictions['prediction']).count() / train_predictions.count()
accuracy_test = test_predictions.filter(test_predictions['Delay'] == test_predictions['prediction']).count() / test_predictions.count()
# Calculate the area under the ROC curve (AUC)
auc_train = evaluator.evaluate(train_predictions, {evaluator.metricName: "areaUnderROC"})
auc_test = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})

print("Training data:")
print(f"Best Model Accuracy: {accuracy_train}")
print(f"Area Under ROC: {auc_train}")
print("Test data:")
print(f"Best Model Accuracy: {accuracy_test}")
print(f"Area Under ROC: {auc_test}")

Training data:
Best Model Accuracy: 0.8439767779390421
Area Under ROC: 0.8314269527999372
Test data:
Best Model Accuracy: 0.8454545454545455
Area Under ROC: 0.8295454545454546


In [82]:
# best model's parameters
best_regParam = bestmodel._java_obj.getRegParam()
best_maxIter = bestmodel._java_obj.getMaxIter()
print(f"Best Reg Param: {best_regParam} \nBest Max Iter: {best_maxIter}")

Best Reg Param: 0.01 
Best Max Iter: 20


In [ ]:
cv_records['linear support vector machine (early stop)'] = [accuracy_train, auc_train, accuracy_test, auc_test]

## 2.Feature importances analysis

In [10]:
# Set a threshold
threshold = 0.001

### 2.1 Logistic Regression


In [16]:
# Assuming model is your trained LogisticRegressionModel
coefficients = lr_Model.coefficients
intercept = lr_Model.intercept

In [17]:
lr_coef = list(map(lambda row: abs(row), list(coefficients)))

In [18]:
# Create a list of tuples containing the original index and the value
indexed_list = [(index, value) for index, value in enumerate(list(coefficients))]

# Sort the indexed list based on the values
sorted_indexed_list = sorted(indexed_list, key=lambda x: x[1])

# Extract the indices after sorting
sorted_indices = [index for index, _ in sorted_indexed_list]

In [19]:
# Get the top 20 features with the higher coefficients
lr_top20_ind = [i<20 for i in sorted_indices]

# Use list comprehension to filter column names based on boolean values
lr_important_features = [col_name for col_name, bool_val in zip(sampled_df.columns, lr_top20_ind) if bool_val]

print(lr_important_features)  # Output: ['id', 'age']

['arrival_airport_code_bit_0', 'arrival_airport_code_bit_3', 'arrival_airport_code_bit_5', 'airline_code_bit_0', 'airline_code_bit_3', 'airline_code_bit_4', 'airline_code_bit_5', 'scheduled_aircraft_type_bit_0', 'scheduled_aircraft_type_bit_1', 'scheduled_aircraft_type_bit_2', 'scheduled_aircraft_type_bit_3', 'scheduled_aircraft_type_bit_4', 'scheduled_aircraft_type_bit_5', 'arrival_airport_bit_7', 'arrival_airport_bit_8', 'aircraft_id_bit_0', 'aircraft_id_bit_1', 'aircraft_id_bit_2', 'aircraft_id_bit_7', 'aircraft_id_bit_8']


### 2.2 Decision Tree

In [21]:
# Get feature importances
dt_feature_importances = dt_model.featureImportances

# Print feature importances
# print("Feature Importances:")
# for feature, importance in zip(feature_columns, feature_importances):
#     print(f"{feature}: {importance}")

# Optionally, select the most important features based on a threshold
threshold = 0.001
dt_important_features = [feature for feature, importance in zip(feature_columns, dt_feature_importances) if importance >= threshold]
print("Important Features:", dt_important_features)

Important Features: ['departure_airport_avg_bearing', 'arrival_airport_max_temperature']


The feature importance shows that it cannot get the important feature than label itself for Decision Trees model.

### 2.3 Random Forest Classifier

In [29]:
# Get feature importances
rf_feature_importances = rf_model.featureImportances

# Print feature importances
# print("Feature Importances:")
# for feature, importance in zip(feature_columns, feature_importances):
#     print(f"{feature}: {importance}")

# Optionally, select the most important features based on a threshold
threshold = 0.01
rf_important_features = [feature for feature, importance in zip(feature_columns, rf_feature_importances) if importance >= threshold]
print("Important Features:")
rf_important_features

Important Features:


['arrival_airport_avg_bearing',
 'arrival_airport_min_temperature',
 'arrival_airport_max_knots',
 'arrival_airport_max_temperature',
 'scheduled_air_time',
 'scheduled_block_time',
 'arrival_airport_code_bit_1',
 'airline_code_bit_2',
 'departure_airport_bit_0',
 'departure_airport_bit_2',
 'departure_airport_bit_8',
 'arrival_airport_bit_2',
 'arrival_airport_bit_5',
 'arrival_airport_bit_7',
 'aircraft_id_bit_1',
 'legacy_route_bit_12',
 'legacy_route_bit_13']

#### 2.3.1 Important freature selection

In [ ]:
# Use important features for further analysis or model training
selected_data = sampled_df.select(rf_important_features + [target_column])

In [30]:
sampled_df = selected_data
feature_df = sampled_df.drop('Delay')

In [31]:
######## Process the data & do train-test split #######

# Define feature columns
feature_columns = feature_df.columns  # Assuming the last column is the target

# Define the target column
target_column = 'Delay'

# Assemble features into a single vector column
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
#sampled_df_model = assembler.transform(sampled_df)
# Define a simple pipeline
pipeline = Pipeline(stages=[assembler])

# Fit the pipeline to the data
pipelineModel = pipeline.fit(sampled_df)

# Transform the data with the pipeline
sampled_df_transformed = pipelineModel.transform(sampled_df)
# Select features and target column for modeling
sampled_df_model = sampled_df_transformed.select("features", target_column)

# Split data into training and test sets
train_df, test_df = sampled_df_model.randomSplit([train_split, test_split])

In [32]:
######## train rf again ########
# Train a RandomForestClassifier model
rf = RandomForestClassifier(labelCol=target_column, featuresCol="features")
rf_model = rf.fit(train_df)

######## get the accuracy & AUC for model trained on important features ########
# Evaluate the model on the training set by accuracy
train_predictions = rf_model.transform(train_df)
evaluator_acc = MulticlassClassificationEvaluator(labelCol=target_column, predictionCol="prediction", metricName="accuracy")
train_accuracy = evaluator_acc.evaluate(train_predictions)
print(f"Training Accuracy: {train_accuracy}")

# Evaluate the model on the test set by accuracy
test_predictions = rf_model.transform(test_df)
test_accuracy = evaluator_acc.evaluate(test_predictions)
print(f"Test Accuracy: {test_accuracy}")

# Calculate the area under the ROC curve (AUC)
evaluator_auc = BinaryClassificationEvaluator(labelCol=target_column, rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = evaluator_auc.evaluate(test_predictions)
print(f"Area Under ROC: {auc}")

Training Accuracy: 1.0
Test Accuracy: 1.0
Area Under ROC: 1.0


After select those most important features, we got the train acc, test acc, and test auc all being 1. It is good, but it become 1 might not just because of the selection of the important features. It **might also caused by the new train-test split** after we select those important features.

### 2.4 Gradient Boosted Decsion Tree

In [35]:
# Get feature importances
gbt_feature_importances = gbt_model.featureImportances

# Print feature importances
# print("Feature Importances:")
# for feature, importance in zip(feature_columns, feature_importances):
#     print(f"{feature}: {importance}")

# Optionally, select the most important features based on a threshold
threshold = 0.01
gbt_important_features = [feature for feature, importance in zip(feature_columns, gbt_feature_importances) if importance >= threshold]
print("Important Features:", gbt_important_features)

Important Features: ['arrival_airport_avg_bearing', 'arrival_airport_avg_knots', 'scheduled_air_time', 'airline_code_bit_2', 'departure_airport_bit_2', 'departure_airport_bit_8', 'arrival_airport_bit_2', 'arrival_airport_bit_5', 'legacy_route_bit_13']


The feature importance shows that it cannot get the important feature than label itself for Gradient Boosted Decision Trees model.

In [36]:
# Optionally, select the most important features based on a threshold
threshold = 0.001
gbt_important_features = [feature for feature, importance in zip(feature_columns, gbt_feature_importances) if importance >= threshold]
print("Important Features:", gbt_important_features)

Important Features: ['arrival_airport_avg_bearing', 'arrival_airport_avg_knots', 'arrival_airport_min_temperature', 'scheduled_air_time', 'airline_code_bit_2', 'departure_airport_bit_2', 'departure_airport_bit_8', 'arrival_airport_bit_2', 'arrival_airport_bit_5', 'legacy_route_bit_13']
